In [ ]:
tweet = '''abandon @OutofAlcatraz I am diagnosed no with borderline personality disorder, obsessive compulsive disorder, post traumatic stress disorder, major depression and ed. I am fucked up too. Who cares? I care. I accept the situation and atleast give some shots to fight them'''

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [ ]:
import re
import nltk

def clean_tweet(tweet):
  tweet = re.sub(r"\d+", "", tweet)
  
  tweet = tweet.replace(".", "")
  tweet = tweet.replace("(", "")
  tweet = tweet.replace(")", "")
  tweet = tweet.replace("'m", " am")
  tweet = tweet.replace("'s", " is")
  tweet = tweet.replace("'ve", " have")
  tweet = tweet.replace("n't", " not")
  tweet = tweet.replace("'re", " are")
  tweet = tweet.replace("'d", " would")
  tweet = tweet.replace("'ll", " will")
  tweet = tweet.replace("\r", " ")
  tweet = tweet.replace("\n", " ")
  tweet = tweet.strip().lower()
  
  tweet = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", tweet)
  return tweet

In [ ]:
import re
import nltk

def find_words_frequency(tweet, words_dict):
  tweet_clean = clean_tweet(tweet)
  token = re.findall('\w+', tweet_clean)
  words_freq = nltk.FreqDist(token)
  total_freq = 0
  for item in words_dict:
    item_freq = words_freq.freq(item)
    total_freq += item_freq
  return total_freq

In [ ]:
def absolutist_words_metric(tweet):
  tweet_clean = clean_tweet(tweet)
  absolutist_words_dict = ["absolutely", "all", "always", "complete", "completely", "constant", "constantly", "definitely", "entire",
                           "ever", "every", "everyone", "everything", "full", "must", "never", "nothing", "totally", "whole"]
  return find_words_frequency(tweet_clean, absolutist_words_dict)

In [ ]:
def first_pronouns_metric(tweet):
  tweet_clean = clean_tweet(tweet)
  first_person_pronoun_dict = ["i", "me", "my", "mine", "we", "us", "our", "ours"]
  return find_words_frequency(tweet_clean, first_person_pronoun_dict)

In [ ]:
def second_third_pronouns_metric(tweet):
  tweet_clean = clean_tweet(tweet)
  second_third_person_pronoun_dict = ["you", "your", "yours",
                                      "he", "she", "it", "him", "her", "his", "its", "hers",
                                      "they", "them", "their", "theirs"]
  return find_words_frequency(tweet_clean, second_third_person_pronoun_dict)

In [ ]:
from textblob import TextBlob

def TextBlob_metrics(tweet):
  tweet_clean = clean_tweet(tweet)
  blob = TextBlob(tweet_clean)
  for sentence in blob.sentences:
    polarity = sentence.sentiment.polarity
    subjectivity = sentence.sentiment.subjectivity
  return polarity, subjectivity

In [ ]:
import csv

def create_anew_dict():
  anew_dict = {}
  with open('/content/EnglishShortened.csv', mode='r') as infile:
    reader = csv.reader(infile)
    anew_dict = {rows[0]:[rows[1], rows[2], rows[3]] for rows in reader}
  return anew_dict

In [ ]:
from nltk.stem import WordNetLemmatizer

def lemmatize_tweet(tweet_clean):
  token = re.findall('\w+', tweet_clean)
  lemmatized_words = []
  lemmatizer = WordNetLemmatizer()
  for word in token:
    lemmatized_words.append(lemmatizer.lemmatize(word))
  return lemmatized_words

In [ ]:
def anew_metric(tweet):
  anew_dict = create_anew_dict()
  valence, arousal, dominance = 0, 0, 0
   
  tweet_clean = clean_tweet(tweet)
  tweet_words = lemmatize_tweet(tweet_clean)
  N_words_total  = len(tweet_words) 
   
  for index in range(N_words_total):
    # check for negation in 3 words before current word
    j = index-1
    neg = False
    while j >= 0 and j >= index-3:
      if tweet_words[j] == 'not' or tweet_words[j] == 'no':
        neg = True
        break
      j -= 1

    # search for lemmatized word in ANEW
    if tweet_words[index] in anew_dict.keys():
      if neg:
        valence += float(anew_dict[tweet_words[index]][0])
        arousal += float(anew_dict[tweet_words[index]][1])
        dominance +=  float(anew_dict[tweet_words[index]][2])
      else:
        valence += (10 - float(anew_dict[tweet_words[index]][0]))
        arousal += (10 - float(anew_dict[tweet_words[index]][1]))
        dominance += (10 - float(anew_dict[tweet_words[index]][2]))

  return valence/N_words_total, arousal/N_words_total, dominance/N_words_total

In [ ]:
import nltk
import re

def pronominalisation_index(tweet):
  tweet_clean = clean_tweet(tweet)
  tokens = re.findall('\w+', tweet_clean)
  tags = nltk.pos_tag(tokens, tagset='universal')
  tags_freq = nltk.FreqDist(tag for (word, tag) in tags)
  return tags_freq['PRON']/tags_freq['NOUN']


In [ ]:
import nltk
import re

def readiness_to_action_index(tweet):
  tweet_clean = clean_tweet(tweet)
  tokens = re.findall('\w+', tweet_clean)
  tags = nltk.pos_tag(tokens, tagset='universal')
  tags_freq = nltk.FreqDist(tag for (word, tag) in tags)
  return tags_freq["VERB"]/tags_freq["NOUN"]

In [ ]:
from nltk.tokenize import sent_tokenize
import string

def punctuation_metric(tweet):
  count = lambda l1,l2: sum([1 for x in l1 if x in l2])
  num_punct = count(tweet,set(string.punctuation))          
  num_sentences = len(sent_tokenize(tweet))
  return num_punct/num_sentences

In [ ]:
def get_tweet_vector(tweet):
  weight_abolutist_metric = 4
  weight_pronouns_metric = 2
  weight_textblob = 1
  weight_anew_metric = 1
  weight_pos_metric = 1

  absolutist = [absolutist_words_metric(tweet)]*weight_abolutist_metric
  pronouns = [first_pronouns_metric(tweet)]*weight_pronouns_metric + [second_third_pronouns_metric(tweet)]*weight_pronouns_metric
  textblob = [TextBlob_metrics(tweet)[0]]*weight_textblob + [TextBlob_metrics(tweet)[1]]*weight_textblob
  anew = [anew_metric(tweet)[0]]*weight_anew_metric + [anew_metric(tweet)[1]]*weight_anew_metric + [anew_metric(tweet)[2]]*weight_anew_metric
  pos = [pronominalisation_index(tweet)]*weight_pos_metric + [readiness_to_action_index(tweet)]*weight_pos_metric + [punctuation_metric(tweet)]*weight_pos_metric

  tweet_vector = absolutist + pronouns + textblob + anew + pos
  
  return tweet_vector               

In [ ]:
get_tweet_vector(tweet)

[0.0,
 0.0,
 0.0,
 0.0,
 0.09523809523809523,
 0.09523809523809523,
 0.023809523809523808,
 0.023809523809523808,
 -0.26875,
 0.6,
 2.6204761904761904,
 2.5438095238095237,
 2.566190476190476,
 0.14285714285714285,
 0.7142857142857143,
 1.6]